# Chapter 11: Sequence-to-sequence Learning (Part 1)

Bab ini membahas dasar-dasar arsitektur *Sequence-to-Sequence* (Seq2Seq), yang menjadi pondasi utama bagi teknologi *Neural Machine Translation* (NMT) atau penerjemahan mesin.

---

## 11.1 Memahami Data Machine Translation
Dalam tugas translasi, model harus memetakan sekuens bahasa sumber ke sekuens bahasa target.
* **Dataset**: Kalimat biasanya diurutkan berdasarkan panjangnya untuk efisiensi *padding*.
* **Arsitektur Utama**: Terdiri dari **Encoder** yang memproses teks sumber dan **Decoder** yang menghasilkan teks target.
* **Context Vector**: Encoder meringkas seluruh informasi dari kalimat sumber ke dalam satu vektor (disebut *context vector* atau *thought vector*) yang kemudian menjadi modal bagi Decoder untuk mulai menerjemahkan.



---

## 11.2 TextVectorization Layer
Dalam TensorFlow 2, proses pengolahan teks disederhanakan melalui layer `TextVectorization`.
* **Fungsi**: Melakukan standarisasi (menghapus tanda baca), tokenisasi (memecah kalimat), dan pemetaan token menjadi integer ID secara otomatis dalam satu langkah yang terintegrasi dengan model.

---

## 11.3 Definisi Encoder, Decoder, dan Model Akhir
Keduanya biasanya dibangun menggunakan unit berulang seperti **GRU** atau **LSTM**.
* **Encoder**: Memproses *sequence source*. *Hidden state* terakhir dari GRU Encoder dijadikan sebagai **Initial State** (kondisi awal) bagi GRU Decoder.
* **Decoder**: Menerima target token dan memprediksi token berikutnya berdasarkan informasi dari Encoder dan token-token yang sudah dihasilkan sebelumnya.

---

## 11.4 Teknik Pelatihan: Teacher Forcing
Dalam fase latihan, model menggunakan teknik **Teacher Forcing** untuk mempercepat konvergensi.
* **Konsep**: Alih-alih memberikan prediksi Decoder dari langkah sebelumnya sebagai input saat ini, kita langsung memberikan **Target Asli** (kata yang benar) kepada Decoder agar ia tetap pada jalur yang benar selama proses belajar.

---

## 11.5 Inference Model (Recursive Decoding)
Saat model digunakan untuk prediksi nyata (*inference*), target asli tidak tersedia.
* **Proses Rekursif**: Decoder beroperasi secara mandiri. Prediksi pada waktu $t$ digunakan kembali sebagai input pada waktu $t+1$.
* Proses ini berlanjut terus-menerus hingga model menghasilkan token `<EOS>` (*End of Sequence*).

---

## 11.6 Metrik Evaluasi: Accuracy & BLEU
Evaluasi teks tidak cukup hanya menggunakan akurasi piksel atau label sederhana.
* **BLEU (Bilingual Evaluation Understudy)**: Metrik standar untuk tugas Seq2Seq. BLEU menghitung seberapa mirip hasil translasi mesin dengan hasil translasi manusia berdasarkan kesamaan N-gram.
* **Implementasi**: Karena TensorFlow menyimpan string dalam format byte, diperlukan konversi menggunakan NumPy sebelum menghitung skor BLEU:
  * Gunakan `translations_in_bytes.numpy().astype(np.bytes_)`.
  * Konversi array byte tersebut kembali ke format string Python untuk diproses oleh *evaluator*.

---

## 11.7 Tip Implementasi di Colab
Untuk membangun Encoder-Decoder yang kuat, pastikan kamu mengembalikan *state* dari Encoder:

```python
# Contoh struktur sederhana
encoder_inputs = tf.keras.Input(shape=(None,), dtype="int64")
encoder_embedding = tf.keras.layers.Embedding(vocab_size, 256)(encoder_inputs)
# return_state=True sangat penting untuk mendapatkan Context Vector
encoder_outputs, encoder_state = tf.keras.layers.GRU(512, return_state=True)(encoder_embedding)

decoder_inputs = tf.keras.Input(shape=(None,), dtype="int64")
# Gunakan encoder_state sebagai initial_state decoder
decoder_gru = tf.keras.layers.GRU(512, return_sequences=True)(decoder_inputs, initial_state=encoder_state)